# Lab 3: Function Calling System

**Week 3 - Advanced Prompting & OpenAI API**

**Provided by:** ADC ENGINEERING & CONSULTING LTD

## Objectives

In this lab, you will:
- Understand OpenAI function calling mechanism
- Define function schemas correctly
- Implement tool-augmented assistants
- Handle function call execution safely
- Build multi-tool systems
- Chain multiple function calls
- Create autonomous agents with tools
- Implement error handling for function calls

## Prerequisites

- Completed Week 3 Labs 1 & 2
- Understanding of JSON schemas
- Python functions and decorators
- OpenAI API key configured
- Python 3.9+

## Setup and Installation

In [ ]:
# Install required packages
!pip install openai python-dotenv tiktoken pydantic requests --quiet

In [ ]:
import os
import json
import inspect
from typing import List, Dict, Optional, Any, Callable, get_type_hints
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum

from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✓ Setup complete!")

## Part 1: Understanding Function Calling

Function calling allows the model to generate structured calls to external tools/APIs.

### How It Works:

1. **Define functions**: Specify available functions with JSON schemas
2. **Send to API**: Include function definitions in the request
3. **Model decides**: LLM decides which function(s) to call
4. **Execute function**: Your code executes the function
5. **Return results**: Send results back to the model
6. **Model responds**: LLM incorporates results into response

Let's start with a simple example:

In [ ]:
def get_current_weather(location: str, unit: str = "celsius") -> dict:
    """
    Get the current weather for a location.
    
    Args:
        location: City name, e.g. "London"
        unit: Temperature unit ("celsius" or "fahrenheit")
    
    Returns:
        Weather information
    """
    # This is a mock function - in reality, you'd call a weather API
    weather_data = {
        "London": {"temperature": 15, "condition": "cloudy"},
        "New York": {"temperature": 22, "condition": "sunny"},
        "Tokyo": {"temperature": 18, "condition": "rainy"}
    }
    
    location_key = location.split(",")[0].strip()
    data = weather_data.get(location_key, {"temperature": 20, "condition": "unknown"})
    
    return {
        "location": location,
        "temperature": data["temperature"],
        "unit": unit,
        "condition": data["condition"]
    }

# Define the function schema for OpenAI
weather_function = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit"
                }
            },
            "required": ["location"]
        }
    }
}

# Test function calling
def test_function_calling():
    """Test basic function calling."""
    messages = [
        {"role": "user", "content": "What's the weather like in London?"}
    ]
    
    # First API call with function definitions
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=[weather_function],
        tool_choice="auto"  # Let model decide
    )
    
    response_message = response.choices[0].message
    
    print("="*80)
    print("BASIC FUNCTION CALLING")
    print("="*80)
    print(f"\nUser: {messages[0]['content']}")
    
    # Check if model wants to call a function
    if response_message.tool_calls:
        print(f"\nModel wants to call function: {response_message.tool_calls[0].function.name}")
        
        # Execute the function
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"Function arguments: {function_args}")
        
        # Call the actual function
        function_response = get_current_weather(**function_args)
        print(f"Function result: {function_response}")
        
        # Add function result to messages
        messages.append(response_message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(function_response)
        })
        
        # Second API call with function result
        final_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        
        print(f"\nFinal response: {final_response.choices[0].message.content}")
    else:
        print(f"\nDirect response: {response_message.content}")

test_function_calling()

### Exercise 1.1: Create Your First Function

Define and test a simple function with the API:

In [ ]:
# TODO: Create a function and its schema

def calculate_tip(bill_amount: float, tip_percentage: float = 15.0) -> dict:
    """
    TODO: Implement tip calculator
    
    Args:
        bill_amount: Total bill amount
        tip_percentage: Tip percentage (default 15%)
    
    Returns:
        Dict with tip amount and total
    """
    pass

# TODO: Define the function schema
tip_calculator_function = {
    "type": "function",
    "function": {
        "name": "calculate_tip",
        "description": "Calculate tip amount and total bill",
        "parameters": {
            # TODO: Add parameters schema
        }
    }
}

# TODO: Test your function
# messages = [{"role": "user", "content": "Calculate a 20% tip on a $50 bill"}]
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
#     tools=[tip_calculator_function],
#     tool_choice="auto"
# )
# Process the response and execute function...

## Part 2: Function Schema Builder

Automatically generate function schemas from Python functions.

In [ ]:
class FunctionSchemaBuilder:
    """
    Automatically build OpenAI function schemas from Python functions.
    """
    
    @staticmethod
    def python_type_to_json_type(py_type: type) -> str:
        """Convert Python type to JSON schema type."""
        type_mapping = {
            str: "string",
            int: "integer",
            float: "number",
            bool: "boolean",
            list: "array",
            dict: "object"
        }
        return type_mapping.get(py_type, "string")
    
    @staticmethod
    def build_schema(func: Callable) -> dict:
        """
        Build OpenAI function schema from Python function.
        
        Args:
            func: Python function to convert
        
        Returns:
            OpenAI function schema
        """
        # Get function signature
        sig = inspect.signature(func)
        type_hints = get_type_hints(func)
        
        # Extract docstring
        doc = inspect.getdoc(func) or ""
        
        # Build parameters schema
        properties = {}
        required = []
        
        for param_name, param in sig.parameters.items():
            if param_name == "self":
                continue
                
            param_type = type_hints.get(param_name, str)
            
            # Get origin type for generics like List[str]
            origin_type = getattr(param_type, "__origin__", param_type)
            
            param_schema = {
                "type": FunctionSchemaBuilder.python_type_to_json_type(origin_type)
            }
            
            # Try to extract description from docstring
            # (Simple extraction - could be more sophisticated)
            if param_name in doc:
                lines = doc.split("\n")
                for i, line in enumerate(lines):
                    if param_name in line and ":" in line:
                        desc = line.split(":", 1)[1].strip()
                        param_schema["description"] = desc
                        break
            
            properties[param_name] = param_schema
            
            # Check if parameter is required (no default value)
            if param.default == inspect.Parameter.empty:
                required.append(param_name)
        
        # Build complete schema
        schema = {
            "type": "function",
            "function": {
                "name": func.__name__,
                "description": doc.split("\n\n")[0] if doc else func.__name__,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                    "required": required
                }
            }
        }
        
        return schema

# Test the schema builder
def search_database(query: str, limit: int = 10, include_metadata: bool = False) -> list:
    """
    Search the database for matching records.
    
    Args:
        query: Search query string
        limit: Maximum number of results to return
        include_metadata: Whether to include metadata in results
    
    Returns:
        List of matching records
    """
    # Mock implementation
    return [{"id": i, "text": f"Result {i} for '{query}'"} for i in range(min(limit, 3))]

# Build schema automatically
schema = FunctionSchemaBuilder.build_schema(search_database)

print("="*80)
print("AUTO-GENERATED FUNCTION SCHEMA")
print("="*80)
print(json.dumps(schema, indent=2))

# Test it works
messages = [{"role": "user", "content": "Search for 'Python tutorials' in the database, limit to 5 results"}]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=[schema],
    tool_choice="auto"
)

if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    print(f"\n✓ Model called function: {tool_call.function.name}")
    print(f"✓ Arguments: {tool_call.function.arguments}")

### Exercise 2.1: Build Schemas for Multiple Functions

Use the schema builder for a set of utility functions:

In [ ]:
# TODO: Create utility functions and auto-generate their schemas

def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict:
    """
    TODO: Convert amount from one currency to another
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., USD)
        to_currency: Target currency code (e.g., EUR)
    
    Returns:
        Conversion result
    """
    pass

def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> dict:
    """
    TODO: Calculate distance between two coordinates
    
    Args:
        lat1: Latitude of first point
        lon1: Longitude of first point
        lat2: Latitude of second point
        lon2: Longitude of second point
    
    Returns:
        Distance in kilometers
    """
    pass

# TODO: Generate schemas
# builder = FunctionSchemaBuilder()
# currency_schema = builder.build_schema(convert_currency)
# distance_schema = builder.build_schema(calculate_distance)

# TODO: Test with the API

## Part 3: Function Registry and Executor

Build a system to register and execute functions safely.

In [ ]:
class FunctionRegistry:
    """
    Registry for managing callable functions.
    """
    
    def __init__(self):
        """Initialize function registry."""
        self.functions: Dict[str, Callable] = {}
        self.schemas: List[dict] = []
    
    def register(self, func: Callable, schema: Optional[dict] = None):
        """
        Register a function.
        
        Args:
            func: Function to register
            schema: Optional pre-built schema (will auto-generate if not provided)
        """
        func_name = func.__name__
        self.functions[func_name] = func
        
        # Generate schema if not provided
        if schema is None:
            schema = FunctionSchemaBuilder.build_schema(func)
        
        self.schemas.append(schema)
        print(f"✓ Registered function: {func_name}")
    
    def get_function(self, name: str) -> Optional[Callable]:
        """Get function by name."""
        return self.functions.get(name)
    
    def get_schemas(self) -> List[dict]:
        """Get all function schemas."""
        return self.schemas
    
    def execute(self, function_name: str, arguments: dict) -> Any:
        """
        Execute a registered function safely.
        
        Args:
            function_name: Name of function to execute
            arguments: Function arguments
        
        Returns:
            Function result
        """
        func = self.get_function(function_name)
        
        if func is None:
            raise ValueError(f"Function '{function_name}' not found in registry")
        
        try:
            result = func(**arguments)
            return result
        except Exception as e:
            return {
                "error": str(e),
                "function": function_name,
                "arguments": arguments
            }

class FunctionCallingAgent:
    """
    Agent that can call functions to answer questions.
    """
    
    def __init__(
        self,
        registry: FunctionRegistry,
        model: str = "gpt-3.5-turbo",
        max_iterations: int = 5
    ):
        """
        Initialize agent.
        
        Args:
            registry: Function registry
            model: OpenAI model to use
            max_iterations: Max function calling iterations
        """
        self.registry = registry
        self.model = model
        self.max_iterations = max_iterations
    
    def chat(self, user_message: str, verbose: bool = True) -> str:
        """
        Chat with the agent, allowing function calls.
        
        Args:
            user_message: User's message
            verbose: Print intermediate steps
        
        Returns:
            Agent's response
        """
        messages = [{"role": "user", "content": user_message}]
        
        if verbose:
            print(f"\n{'='*80}")
            print(f"User: {user_message}")
            print(f"{'='*80}")
        
        for iteration in range(self.max_iterations):
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.registry.get_schemas(),
                tool_choice="auto"
            )
            
            response_message = response.choices[0].message
            
            # Check if model wants to call functions
            if not response_message.tool_calls:
                # No more function calls, return response
                if verbose:
                    print(f"\nAgent: {response_message.content}")
                return response_message.content
            
            # Process function calls
            messages.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                if verbose:
                    print(f"\n🔧 Calling: {function_name}({json.dumps(function_args)})")
                
                # Execute function
                function_result = self.registry.execute(function_name, function_args)
                
                if verbose:
                    print(f"📊 Result: {function_result}")
                
                # Add function result to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(function_result)
                })
        
        # Max iterations reached
        return "I apologize, but I couldn't complete the task within the allowed steps."

# Test the system
registry = FunctionRegistry()

# Register functions
registry.register(get_current_weather)
registry.register(search_database)

# Create agent
agent = FunctionCallingAgent(registry)

# Test queries
test_queries = [
    "What's the weather in Tokyo?",
    "Search the database for 'machine learning' topics",
    "What's the weather in New York and can you search for 'weather data' in the database?"
]

for query in test_queries:
    response = agent.chat(query, verbose=True)
    print("\n" + "-"*80 + "\n")

### Exercise 3.1: Build a Multi-Function Agent

Create an agent with multiple tools:

In [ ]:
# TODO: Create multiple utility functions and build an agent

def get_current_time(timezone: str = "UTC") -> dict:
    """
    TODO: Get current time in a timezone
    
    Args:
        timezone: Timezone name (e.g., "UTC", "America/New_York")
    
    Returns:
        Current time information
    """
    pass

def calculate_mortgage(
    principal: float,
    annual_rate: float,
    years: int
) -> dict:
    """
    TODO: Calculate monthly mortgage payment
    
    Args:
        principal: Loan amount
        annual_rate: Annual interest rate (as percentage, e.g., 3.5)
        years: Loan term in years
    
    Returns:
        Payment calculation
    """
    pass

def translate_text(text: str, target_language: str) -> dict:
    """
    TODO: Translate text to target language (mock)
    
    Args:
        text: Text to translate
        target_language: Target language code (e.g., "es", "fr")
    
    Returns:
        Translation result
    """
    pass

# TODO: Register functions and create agent
# my_registry = FunctionRegistry()
# my_registry.register(get_current_time)
# my_registry.register(calculate_mortgage)
# my_registry.register(translate_text)

# my_agent = FunctionCallingAgent(my_registry)

# TODO: Test with complex queries
# response = my_agent.chat("What time is it in Tokyo and translate 'Hello' to Spanish?")

## Part 4: Advanced Function Patterns

Implement advanced patterns like parallel calls and chained execution.

In [ ]:
@dataclass
class FunctionCallResult:
    """Result of a function call."""
    function_name: str
    arguments: dict
    result: Any
    success: bool
    error: Optional[str] = None
    execution_time: float = 0.0

class AdvancedFunctionAgent:
    """
    Advanced agent with parallel execution and chaining.
    """
    
    def __init__(
        self,
        registry: FunctionRegistry,
        model: str = "gpt-3.5-turbo"
    ):
        """
        Initialize advanced agent.
        
        Args:
            registry: Function registry
            model: OpenAI model to use
        """
        self.registry = registry
        self.model = model
        self.call_history: List[FunctionCallResult] = []
    
    def execute_parallel(
        self,
        function_calls: List[tuple[str, dict]]
    ) -> List[FunctionCallResult]:
        """
        Execute multiple function calls in parallel.
        
        Args:
            function_calls: List of (function_name, arguments) tuples
        
        Returns:
            List of results
        """
        results = []
        
        for func_name, args in function_calls:
            import time
            start_time = time.time()
            
            try:
                result = self.registry.execute(func_name, args)
                success = True
                error = None
            except Exception as e:
                result = None
                success = False
                error = str(e)
            
            execution_time = time.time() - start_time
            
            call_result = FunctionCallResult(
                function_name=func_name,
                arguments=args,
                result=result,
                success=success,
                error=error,
                execution_time=execution_time
            )
            
            results.append(call_result)
            self.call_history.append(call_result)
        
        return results
    
    def chat_with_parallel_execution(
        self,
        user_message: str,
        max_iterations: int = 5,
        verbose: bool = True
    ) -> str:
        """
        Chat with parallel function execution support.
        
        Args:
            user_message: User's message
            max_iterations: Max iterations
            verbose: Print steps
        
        Returns:
            Agent's response
        """
        messages = [{"role": "user", "content": user_message}]
        
        if verbose:
            print(f"\n{'='*80}")
            print(f"User: {user_message}")
            print(f"{'='*80}")
        
        for iteration in range(max_iterations):
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.registry.get_schemas(),
                tool_choice="auto"
            )
            
            response_message = response.choices[0].message
            
            if not response_message.tool_calls:
                if verbose:
                    print(f"\nAgent: {response_message.content}")
                return response_message.content
            
            messages.append(response_message)
            
            # Collect all function calls for parallel execution
            function_calls = []
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                function_calls.append((function_name, function_args))
            
            if verbose:
                print(f"\n🚀 Executing {len(function_calls)} function(s) in parallel...")
            
            # Execute all functions in parallel
            results = self.execute_parallel(function_calls)
            
            # Add results back to messages
            for tool_call, result in zip(response_message.tool_calls, results):
                if verbose:
                    status = "✓" if result.success else "✗"
                    print(f"{status} {result.function_name}: {result.result} ({result.execution_time:.3f}s)")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result.result if result.success else {"error": result.error})
                })
        
        return "Maximum iterations reached."
    
    def get_execution_stats(self) -> dict:
        """Get statistics about function executions."""
        if not self.call_history:
            return {"message": "No function calls yet"}
        
        total_calls = len(self.call_history)
        successful = sum(1 for call in self.call_history if call.success)
        failed = total_calls - successful
        
        total_time = sum(call.execution_time for call in self.call_history)
        avg_time = total_time / total_calls
        
        # Function usage
        from collections import Counter
        function_counts = Counter(call.function_name for call in self.call_history)
        
        return {
            "total_calls": total_calls,
            "successful": successful,
            "failed": failed,
            "success_rate": f"{(successful/total_calls*100):.1f}%",
            "total_execution_time": f"{total_time:.3f}s",
            "avg_execution_time": f"{avg_time:.3f}s",
            "function_usage": dict(function_counts)
        }

# Test advanced agent
advanced_registry = FunctionRegistry()
advanced_registry.register(get_current_weather)
advanced_registry.register(search_database)

advanced_agent = AdvancedFunctionAgent(advanced_registry)

# Test with query requiring multiple function calls
query = "What's the weather in London, New York, and Tokyo? Also search the database for 'weather patterns'."
response = advanced_agent.chat_with_parallel_execution(query, verbose=True)

# Get execution stats
print("\n" + "="*80)
print("EXECUTION STATISTICS")
print("="*80)
stats = advanced_agent.get_execution_stats()
for key, value in stats.items():
    print(f"{key}: {value}")

### Exercise 4.1: Implement Function Chaining

Build a system where one function's output feeds into another:

In [ ]:
# TODO: Implement function chaining

class FunctionChain:
    """
    Execute functions in sequence, passing outputs as inputs.
    
    TODO: Implement:
    1. Define chain of functions
    2. Execute in sequence
    3. Pass outputs to next function
    4. Handle errors in chain
    5. Support conditional execution
    """
    
    def __init__(self, registry: FunctionRegistry):
        """
        Initialize chain.
        
        Args:
            registry: Function registry
        """
        self.registry = registry
        self.steps: List[dict] = []
    
    def add_step(
        self,
        function_name: str,
        arguments: dict,
        output_mapping: Optional[dict] = None
    ):
        """
        TODO: Add a step to the chain.
        
        Args:
            function_name: Function to call
            arguments: Function arguments (can reference previous outputs)
            output_mapping: How to map output to next step's input
        """
        pass
    
    def execute(self, initial_input: dict) -> dict:
        """
        TODO: Execute the entire chain.
        
        Args:
            initial_input: Initial input data
        
        Returns:
            Final result
        """
        pass

# Usage example:
# chain = FunctionChain(registry)
# chain.add_step("get_current_weather", {"location": "London"})
# chain.add_step("search_database", {"query": "{previous.condition}"})  # Use previous result
# result = chain.execute({})

## Part 5: Error Handling and Validation

Robust error handling for function calling.

In [ ]:
class SafeFunctionRegistry(FunctionRegistry):
    """
    Function registry with validation and error handling.
    """
    
    def __init__(self):
        """Initialize safe registry."""
        super().__init__()
        self.validators: Dict[str, Callable] = {}
    
    def register_with_validator(
        self,
        func: Callable,
        validator: Optional[Callable] = None,
        schema: Optional[dict] = None
    ):
        """
        Register function with optional validator.
        
        Args:
            func: Function to register
            validator: Function to validate arguments before execution
            schema: Optional pre-built schema
        """
        self.register(func, schema)
        
        if validator:
            self.validators[func.__name__] = validator
    
    def validate_arguments(
        self,
        function_name: str,
        arguments: dict
    ) -> tuple[bool, Optional[str]]:
        """
        Validate function arguments.
        
        Args:
            function_name: Function name
            arguments: Arguments to validate
        
        Returns:
            (is_valid, error_message)
        """
        validator = self.validators.get(function_name)
        
        if validator is None:
            return True, None
        
        try:
            validator(arguments)
            return True, None
        except Exception as e:
            return False, str(e)
    
    def execute(self, function_name: str, arguments: dict) -> Any:
        """
        Execute function with validation.
        
        Args:
            function_name: Function name
            arguments: Function arguments
        
        Returns:
            Function result or error
        """
        # Validate arguments first
        is_valid, error_msg = self.validate_arguments(function_name, arguments)
        
        if not is_valid:
            return {
                "error": f"Validation failed: {error_msg}",
                "function": function_name,
                "arguments": arguments
            }
        
        # Execute with error handling
        return super().execute(function_name, arguments)

# Example validators
def validate_weather_args(args: dict):
    """Validate weather function arguments."""
    location = args.get("location", "")
    if len(location) < 2:
        raise ValueError("Location must be at least 2 characters")
    
    unit = args.get("unit", "celsius")
    if unit not in ["celsius", "fahrenheit"]:
        raise ValueError(f"Invalid unit: {unit}")

def validate_search_args(args: dict):
    """Validate search function arguments."""
    query = args.get("query", "")
    if len(query) < 3:
        raise ValueError("Query must be at least 3 characters")
    
    limit = args.get("limit", 10)
    if not isinstance(limit, int) or limit < 1 or limit > 100:
        raise ValueError("Limit must be between 1 and 100")

# Test safe registry
safe_registry = SafeFunctionRegistry()
safe_registry.register_with_validator(
    get_current_weather,
    validator=validate_weather_args
)
safe_registry.register_with_validator(
    search_database,
    validator=validate_search_args
)

print("="*80)
print("SAFE FUNCTION EXECUTION WITH VALIDATION")
print("="*80)

# Test valid call
print("\n✓ Valid call:")
result = safe_registry.execute("get_current_weather", {"location": "London", "unit": "celsius"})
print(f"Result: {result}")

# Test invalid call
print("\n✗ Invalid call (bad unit):")
result = safe_registry.execute("get_current_weather", {"location": "London", "unit": "kelvin"})
print(f"Result: {result}")

# Test with agent
safe_agent = FunctionCallingAgent(safe_registry)
response = safe_agent.chat("What's the weather in X?", verbose=True)  # Should fail validation

### Exercise 5.1: Build Comprehensive Validation

Create validators for complex function arguments:

In [ ]:
# TODO: Create functions with comprehensive validation

def book_flight(
    origin: str,
    destination: str,
    date: str,
    passengers: int,
    class_type: str = "economy"
) -> dict:
    """
    TODO: Book a flight (mock)
    
    Args:
        origin: Origin airport code (3 letters)
        destination: Destination airport code (3 letters)
        date: Flight date (YYYY-MM-DD)
        passengers: Number of passengers (1-9)
        class_type: Seat class (economy/business/first)
    
    Returns:
        Booking confirmation
    """
    pass

def validate_flight_booking(args: dict):
    """
    TODO: Validate flight booking arguments
    
    Should check:
    - Airport codes are 3 uppercase letters
    - Date format is correct and in future
    - Passengers between 1-9
    - Class type is valid
    - Origin != destination
    """
    pass

# TODO: Register with validation and test
# safe_registry = SafeFunctionRegistry()
# safe_registry.register_with_validator(book_flight, validate_flight_booking)

# Test with various inputs (valid and invalid)

## Part 6: Building an Autonomous Agent

Create an agent that can use tools to solve complex tasks.

In [ ]:
class AutonomousAgent:
    """
    Autonomous agent that can plan and execute multi-step tasks.
    """
    
    def __init__(
        self,
        registry: FunctionRegistry,
        model: str = "gpt-4",  # Better reasoning with GPT-4
        max_steps: int = 10
    ):
        """
        Initialize autonomous agent.
        
        Args:
            registry: Function registry
            model: OpenAI model to use
            max_steps: Maximum steps to take
        """
        self.registry = registry
        self.model = model
        self.max_steps = max_steps
        self.conversation_history: List[dict] = []
    
    def solve_task(
        self,
        task: str,
        verbose: bool = True
    ) -> dict:
        """
        Solve a complex task autonomously.
        
        Args:
            task: Task description
            verbose: Print reasoning steps
        
        Returns:
            Solution with execution trace
        """
        if verbose:
            print(f"\n{'='*80}")
            print(f"🎯 TASK: {task}")
            print(f"{'='*80}\n")
        
        # System message for autonomous behavior
        system_message = {
            "role": "system",
            "content": """You are an autonomous agent capable of using tools to solve tasks.
            
For each step:
1. Analyze what information you need
2. Decide which tool(s) to use
3. Execute the tool(s)
4. Review the results
5. Determine if task is complete or continue

Be methodical and explain your reasoning."""
        }
        
        self.conversation_history = [
            system_message,
            {"role": "user", "content": task}
        ]
        
        steps_taken = []
        
        for step_num in range(1, self.max_steps + 1):
            if verbose:
                print(f"📍 Step {step_num}/{self.max_steps}")
                print("-" * 80)
            
            # Get model's decision
            response = client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                tools=self.registry.get_schemas(),
                tool_choice="auto"
            )
            
            response_message = response.choices[0].message
            
            # Check if task is complete
            if not response_message.tool_calls:
                if verbose:
                    print(f"\n✅ Task Complete!")
                    print(f"Final Answer: {response_message.content}\n")
                
                return {
                    "success": True,
                    "steps_taken": step_num,
                    "final_answer": response_message.content,
                    "execution_trace": steps_taken
                }
            
            # Execute tool calls
            self.conversation_history.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                if verbose:
                    print(f"🔧 Using tool: {function_name}")
                    print(f"   Arguments: {json.dumps(function_args, indent=2)}")
                
                # Execute function
                try:
                    result = self.registry.execute(function_name, function_args)
                    success = True
                except Exception as e:
                    result = {"error": str(e)}
                    success = False
                
                if verbose:
                    status = "✓" if success else "✗"
                    print(f"{status} Result: {result}\n")
                
                # Record step
                steps_taken.append({
                    "step": step_num,
                    "function": function_name,
                    "arguments": function_args,
                    "result": result,
                    "success": success
                })
                
                # Add result to conversation
                self.conversation_history.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })
        
        # Max steps reached
        if verbose:
            print(f"\n⚠️  Maximum steps ({self.max_steps}) reached without completion\n")
        
        return {
            "success": False,
            "steps_taken": self.max_steps,
            "final_answer": "Task not completed within step limit",
            "execution_trace": steps_taken
        }

# Create tools for the autonomous agent
def get_stock_price(symbol: str) -> dict:
    """Get current stock price (mock)."""
    # Mock data
    prices = {
        "AAPL": 175.50,
        "GOOGL": 140.25,
        "MSFT": 380.75,
        "TSLA": 245.30
    }
    return {
        "symbol": symbol,
        "price": prices.get(symbol.upper(), 100.0),
        "currency": "USD"
    }

def calculate_percentage_change(old_value: float, new_value: float) -> dict:
    """Calculate percentage change."""
    change = ((new_value - old_value) / old_value) * 100
    return {
        "old_value": old_value,
        "new_value": new_value,
        "percentage_change": round(change, 2),
        "direction": "up" if change > 0 else "down" if change < 0 else "unchanged"
    }

def calculate_average(numbers: List[float]) -> dict:
    """Calculate average of numbers."""
    if not numbers:
        return {"error": "Empty list"}
    avg = sum(numbers) / len(numbers)
    return {
        "numbers": numbers,
        "average": round(avg, 2),
        "count": len(numbers)
    }

# Register functions
auto_registry = FunctionRegistry()
auto_registry.register(get_stock_price)
auto_registry.register(calculate_percentage_change)
auto_registry.register(calculate_average)
auto_registry.register(get_current_weather)

# Create autonomous agent
auto_agent = AutonomousAgent(auto_registry, model="gpt-3.5-turbo")  # Use gpt-4 for better reasoning

# Test with complex task
complex_task = """
Compare the stock prices of Apple (AAPL) and Microsoft (MSFT).
Then calculate the average of their prices and tell me which one is performing better.
"""

result = auto_agent.solve_task(complex_task, verbose=True)

print("="*80)
print("TASK SUMMARY")
print("="*80)
print(f"Success: {result['success']}")
print(f"Steps taken: {result['steps_taken']}")
print(f"Final answer: {result['final_answer']}")

### Exercise 6.1: Build a Research Assistant

Create an autonomous agent that can research topics:

In [ ]:
# TODO: Build research assistant with these tools

def web_search(query: str, num_results: int = 5) -> dict:
    """
    TODO: Search the web (mock)
    
    Args:
        query: Search query
        num_results: Number of results
    
    Returns:
        Search results
    """
    pass

def extract_key_points(text: str, num_points: int = 5) -> dict:
    """
    TODO: Extract key points from text (mock)
    
    Args:
        text: Text to analyze
        num_points: Number of key points to extract
    
    Returns:
        Key points
    """
    pass

def summarize_findings(data: List[str]) -> dict:
    """
    TODO: Summarize multiple pieces of information
    
    Args:
        data: List of text to summarize
    
    Returns:
        Summary
    """
    pass

# TODO: Create research assistant agent
# research_registry = FunctionRegistry()
# research_registry.register(web_search)
# research_registry.register(extract_key_points)
# research_registry.register(summarize_findings)

# research_assistant = AutonomousAgent(research_registry)

# TODO: Test with research tasks
# task = "Research the latest developments in quantum computing and summarize the top 3 breakthroughs"
# result = research_assistant.solve_task(task)

## Challenge Projects

### Challenge 1: Multi-Agent System

Build a system with multiple specialized agents:

In [ ]:
class MultiAgentSystem:
    """
    System with multiple specialized agents.
    
    TODO: Implement:
    1. Different agents for different domains (research, data analysis, writing)
    2. Router agent to delegate tasks
    3. Agents can call each other
    4. Collaborative problem solving
    5. Result synthesis from multiple agents
    """
    
    def __init__(self):
        """Initialize multi-agent system."""
        self.agents: Dict[str, AutonomousAgent] = {}
        self.router = None
    
    def add_agent(self, name: str, agent: AutonomousAgent):
        """TODO: Add specialized agent."""
        pass
    
    def route_task(self, task: str) -> str:
        """TODO: Route task to appropriate agent."""
        pass
    
    def solve_collaborative(self, task: str) -> dict:
        """TODO: Solve task using multiple agents."""
        pass

# Usage example:
# system = MultiAgentSystem()
# system.add_agent("researcher", research_agent)
# system.add_agent("analyst", data_analyst_agent)
# system.add_agent("writer", writing_agent)
# result = system.solve_collaborative("Research AI trends and write a report")

### Challenge 2: Function Calling with Memory

Build an agent with persistent memory:

In [ ]:
class MemoryAgent:
    """
    Agent with persistent memory of past interactions.
    
    TODO: Implement:
    1. Store conversation history
    2. Remember function call results
    3. Reference past information in new queries
    4. Memory summarization when context grows large
    5. Forgetting mechanism for old/irrelevant info
    6. Memory search/retrieval
    """
    
    def __init__(self, registry: FunctionRegistry):
        """Initialize agent with memory."""
        self.registry = registry
        self.memory = []
        self.function_call_cache = {}
    
    def remember(self, key: str, value: Any):
        """TODO: Store information in memory."""
        pass
    
    def recall(self, query: str) -> Any:
        """TODO: Retrieve information from memory."""
        pass
    
    def chat_with_memory(self, message: str) -> str:
        """TODO: Chat using memory."""
        pass

# Usage:
# memory_agent = MemoryAgent(registry)
# memory_agent.chat_with_memory("What's the weather in London?")
# memory_agent.chat_with_memory("What was the weather there earlier?")  # References previous

### Challenge 3: Self-Improving Agent

Build an agent that learns from its mistakes:

In [ ]:
class SelfImprovingAgent:
    """
    Agent that learns from execution results.
    
    TODO: Implement:
    1. Track success/failure of function calls
    2. Learn which functions work for which tasks
    3. Adapt strategy based on past results
    4. A/B test different approaches
    5. Build knowledge base of solutions
    6. Suggest new functions to implement
    """
    
    def __init__(self, registry: FunctionRegistry):
        """Initialize self-improving agent."""
        self.registry = registry
        self.performance_history = []
        self.learned_strategies = {}
    
    def record_outcome(self, task: str, strategy: str, success: bool):
        """TODO: Record task outcome."""
        pass
    
    def select_strategy(self, task: str) -> str:
        """TODO: Select best strategy based on learning."""
        pass
    
    def suggest_new_function(self, failed_task: str) -> dict:
        """TODO: Suggest function that would help."""
        pass

# Usage:
# learning_agent = SelfImprovingAgent(registry)
# for task in tasks:
#     result = learning_agent.solve_task(task)
#     learning_agent.record_outcome(task, result['strategy'], result['success'])

## Summary

In this lab, you've learned:

1. ✅ OpenAI function calling fundamentals
2. ✅ Automatic function schema generation
3. ✅ Function registry and safe execution
4. ✅ Advanced patterns (parallel execution, chaining)
5. ✅ Comprehensive error handling and validation
6. ✅ Building autonomous agents with tools
7. ✅ Multi-step task solving
8. ✅ Complex agent architectures

### Key Takeaways

**Function Calling Benefits:**
- Structured outputs from LLMs
- Integration with external systems
- Deterministic tool usage
- Reduced hallucination for factual tasks
- Building compound AI systems

**Best Practices:**

1. **Schema Design:**
   - Clear function descriptions
   - Detailed parameter descriptions
   - Use enums for limited choices
   - Mark required parameters
   - Provide examples in descriptions

2. **Error Handling:**
   - Validate arguments before execution
   - Catch and report errors gracefully
   - Return structured error messages
   - Implement retry logic
   - Set execution timeouts

3. **Safety:**
   - Validate all inputs
   - Sandbox function execution
   - Rate limit function calls
   - Log all executions
   - Implement authentication/authorization

4. **Performance:**
   - Execute functions in parallel when possible
   - Cache function results
   - Set reasonable timeouts
   - Monitor execution time
   - Limit max iterations

### Common Patterns

**1. Single Function:** Simple tool use
**2. Multiple Functions:** Agent with toolbox
**3. Chained Functions:** Output → Input pipeline
**4. Parallel Functions:** Multiple simultaneous calls
**5. Autonomous Agent:** Self-directed task solving

### Next Steps

- Complete the challenge projects
- Build your own tool-augmented assistant
- Integrate real APIs (weather, database, etc.)
- Move on to Week 4: RAG Fundamentals
- Experiment with complex multi-agent systems

**Provided by:** ADC ENGINEERING & CONSULTING LTD